# J1-J2 model

In this tutorial, we use neural quantum states and variational Monte Carlo to study the J1-J2 Heisenberg model.

## System definition

We will work on the 4x4 lattice. Due to the spin conservation in the Heisenberg interactions, we limit the number of both up and down spins to 8.

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import numpy as np
import matplotlib.pyplot as plt
import jax.numpy as jnp
import quantax as qtx

# 4x4 square lattice, 8 spin-up, 8 spin-down
lattice = qtx.sites.Square(4, Nparticles=(8, 8))
N = lattice.Nsites

The Hamiltonian is

$$
H = J_1 \sum_{\braket{ij}} \mathbf{\sigma}_i \cdot \mathbf{\sigma}_j
+ J_2 \sum_{\braket{\braket{ij}}} \mathbf{\sigma}_i \cdot \mathbf{\sigma}_j,
$$

where $\braket{ij}$ and $\braket{\braket{ij}}$ represent nearest and next-nearest neighbors, respectively. 

Quantax provides a helper function to define this Hamiltonian as shown below.

In [4]:
H = qtx.operator.Heisenberg(J=[1, 0.5], n_neighbor=[1, 2], msr=True)

In this function, we define $J_1 = 1$ and $J_2 = 0.5$. The neighbors are determined by the distance of spins in real space. With `msr=True`, the function returns a Hamiltonian corrected by Marshall sign rule (MSR), which is equivalent to the original J1-J2 model but has simpler sign structures.

In this small lattice, one can use exact diagonalization to obtain the reference energy.

In [6]:
E, wf = H.diagonalize()
print(f"Exact ground state energy: {E[0]}")

Exact ground state energy: -33.831693405579394


## Variational state

